# Guessing with Negative Marking


---


## Use random number generator crate 

In [2]:
extern crate rand;
use rand::distributions::{Distribution, Uniform};
use rand::{thread_rng,Rng};
use rand::seq::SliceRandom;

## Generate a set of random responses

In [3]:
fn generate_random_answers(num_questions: usize, num_options: u8, can_skip: bool) -> Vec<u8> {
    let between;
    
    if can_skip {
        // Initialise uniform distribution between 0 and 4
        between = Uniform::new(0_u8, 1 + num_options);
    } else {
        // Initialise uniform distribution between 1 and 4
        between = Uniform::new(1_u8, 1 + num_options);
    }
    
    // Generate 30 "answers"
    thread_rng().sample_iter(&between).take(num_questions).collect::<Vec<u8>>()
}

In [4]:
let a = generate_random_answers(30, 4, false);
println!("{:?}", a);

[4, 2, 3, 2, 3, 4, 1, 3, 3, 1, 3, 3, 2, 2, 2, 2, 3, 4, 2, 3, 4, 3, 1, 1, 3, 3, 2, 1, 2, 4]


## Allocate marks

In [5]:
fn allocate_marks(num_one_mark: usize, num_two_mark: usize) -> Vec<u8> {
    let mut marks = vec![1_u8; num_one_mark];
    let twos = vec![2_u8; num_two_mark];
    
    marks.extend(twos);
    
    let mut rng = thread_rng();
    marks.shuffle(&mut rng);
    
    marks
}

In [6]:
let b = allocate_marks(20, 10);
println!("{:?}", b);

[1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1]


## Create a scenario

In [7]:
fn scenario(num_questions: usize, num_options: u8, num_one_mark: usize, 
            num_two_mark: usize, num_times: usize, penalty: f64, 
            enable_skip: bool) -> f64 
{
    // Ensure penality is positive
    let updated_penalty;
    
    if penalty < 0.0 {
        updated_penalty = -1.0*penalty
    } else {
        updated_penalty = penalty
    }
    
    // Randomly allocate correct answers and marks
    let correct_answers = generate_random_answers(num_questions, num_options, false);
    let mark_allocation = allocate_marks(num_one_mark, num_two_mark);
    
    // Combine answers and marks for easy comparison at next step
    let answer_and_marks = correct_answers.iter().zip(mark_allocation.iter());
    
    // If correct answer, allocate mark. Else, implement penalty
    let total_scores: f64 = (0..num_times).fold(0.0_f64, |scores_sum, _| 
        match answer_and_marks.clone().zip(generate_random_answers(num_questions, num_options, enable_skip)
                                          .iter())
                                      .fold(0.0_f64, |test_sum, values| 
                                          if values.0.0 == values.1 {
                                              test_sum + *values.0.1 as f64
                                          } else if values.1 == &0_u8 {
                                              test_sum 
                                          }
                                          else {
                                              test_sum - updated_penalty
                                          })
        {
            score if score >= 0.0_f64 => scores_sum + score,
            _ => scores_sum,
        }
        );
    
    total_scores/(num_times as f64)
}

## Analysis

In [8]:
let num_questions: usize = 30;
let num_options: u8 = 4;
let num_one_mark: usize = 20;
let num_two_mark: usize = 10;
let num_times: usize = 10000000;

### Simplest case

The simplest case would be without there being negative marking and skipping of questions. 
The expected mean score would be, 
$$
    E(simplest) = \frac{1}{numoptions}*totalscore
$$
where $numoptions$ is the total number of options available and $totalscore$ is highest possible attainable score.

With the experimental setup, this would be $10$.

In [9]:
let penalty: f64 = 0.0;
let enable_skip = false;

let simplest_case = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
println!("No negative marking, no skipping (mean): {}", simplest_case);

No negative marking, no skipping (mean): 10.0006142


### No negative marking - with skipping

Since skipping will mean no marks are awarded, the expected mean will be lower. 
How much lower depends on the probability of skipping a question. 

Here, it is assumed that there is a uniform distribution between entering one of the 4 options and skipping. 
Therefore, there is a $20\%$ chance of getting the correct answer. Hence, an expected score of $8$.

In [10]:
let penalty: f64 = 0.0;
let enable_skip = true;

let no_neg_w_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
println!("No negative marking (mean): {}", no_neg_w_skip);

No negative marking (mean): 7.9983804


### Penalty of 0.25 

In [11]:
let penalty: f64 = 0.25;
let enable_skip = false;

let neg_quart_no_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
let enable_skip = true;
let neg_quart_w_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
println!("Negative marking of 0.5 no skip (mean): {}", neg_quart_no_skip);
println!("Negative marking of 0.5 with skip (mean): {}", neg_quart_w_skip);

Negative marking of 0.5 no skip (mean): 4.594486
Negative marking of 0.5 with skip (mean): 3.752348875


### Penalty of 0.5 - no skipping

In [12]:
let penalty: f64 = 0.5;
let enable_skip = false;

let neg_half_no_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
println!("Negative marking of 0.5 no skip (mean): {}", neg_half_no_skip);

Negative marking of 0.5 no skip (mean): 1.2560947


### Penalty of 0.5 - with skipping

In [13]:
let penalty: f64 = 0.5;
let enable_skip = true;

let neg_half_w_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
println!("Negative marking of 0.5 with skip (mean): {}", neg_half_w_skip);

Negative marking of 0.5 with skip (mean): 1.17329865


### Penalty of 0.75 

In [15]:
let penalty: f64 = 0.75;
let enable_skip = false;

let neg_threequart_no_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
let enable_skip = true;
let neg_threequart_w_skip = scenario(num_questions, num_options, num_one_mark, 
                                num_two_mark, num_times, penalty, enable_skip);
println!("Negative marking of 0.75 no skip (mean): {}", neg_threequart_no_skip);
println!("Negative marking of 0.75 with skip (mean): {}", neg_threequart_w_skip);

Negative marking of 0.75 no skip (mean): 0.242034025
Negative marking of 0.75 with skip (mean): 0.29118515
